# Final project

**Name:** Silvia Ivonne Velázquez Ramírez

**e-mail:** silvia.velazquez7499@alumnos.udg.mx

# Modules

In [23]:
#Modules here
import pandas as pd
import plotly.graph_objs as go
import dash
import dash_core_components as dcc
from dash import html
from dash.dependencies import Input, Output

# Final project

In [32]:
# Code here
# Uploading dataset
df = pd.read_csv("dataset.csv", encoding="latin1")

# Initializing Dash application
app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("Grades dashboard"),
    
    # Filters
    html.Label("Select the calendar:"),
    dcc.Dropdown(
        id='ciclo-selector',
        options=[{'label': ciclo, 'value': ciclo} for ciclo in sorted(df['CICLO'].unique())],
        value=df['CICLO'].unique()[0]
    ),

    html.Br(), html.Br(),
    html.Label("Select the subject:"),
    dcc.Dropdown(
        id='materia-selector',
        options=[{'label': materia, 'value': materia} for materia in sorted(df['MATERIA'].unique())],
        value=df['MATERIA'].unique()[0]
    ),
    
    # Graphics
    dcc.Graph(id='histograma-calificaciones'),
    dcc.Graph(id='grafico-categorias'),
    
    # Metrics
    html.Div(id='metricas-calificaciones')
])

@app.callback(
    [Output('histograma-calificaciones', 'figure'),
     Output('grafico-categorias', 'figure'),
     Output('metricas-calificaciones', 'children')],
    [Input('ciclo-selector', 'value'),
     Input('materia-selector', 'value')]
)
def actualizar_dashboard(ciclo, materia):
    df_filtrado = df[(df['CICLO'] == ciclo) & (df['MATERIA'] == materia)]
    
    # Filter numeric grades
    df_numerico = df_filtrado[pd.to_numeric(df_filtrado['CALIFICACION'], errors='coerce').notna()]
    df_numerico = df_numerico.copy()
    df_numerico['CALIFICACION'] = df_numerico['CALIFICACION'].astype(float)
    #df_numerico['CALIFICACION'] = df_numerico['CALIFICACION'].astype(float)
    
    # Grades histogram
    fig_hist = go.Figure(data=[go.Histogram(x=df_numerico['CALIFICACION'], nbinsx=10)])
    fig_hist.update_layout(title=f'Distribución de Calificaciones ({materia} - {ciclo})', xaxis_title='Calificación', yaxis_title='Frecuencia')
    
    # Category grades count
    conteo_categorias = df_filtrado['CALIFICACION'].value_counts()
    fig_cat = go.Figure(data=[go.Bar(x=conteo_categorias.index, y=conteo_categorias.values)])
    fig_cat.update_layout(title='Distribución de Calificaciones Categóricas', xaxis_title='Tipo de Calificación', yaxis_title='Frecuencia')
    
    # Metrics
    total_alumnos = len(df_filtrado)
    promedio = df_numerico['CALIFICACION'].mean() if not df_numerico.empty else 'N/A'
    metricas = html.Div([
        html.P(f'Total de alumnos: {total_alumnos}'),
        html.P(f'Promedio de calificaciones: {promedio:.2f}' if promedio != 'N/A' else 'Promedio de calificaciones: N/A')
    ])
    
    return fig_hist, fig_cat, metricas

if __name__ == '__main__':
    app.run_server(debug=True)


